In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

In [15]:
data_df = pd.read_csv("output_data/cities.csv")

In [16]:
data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rabo de peixe,37.80,-25.58,73.400,88.0,40.0,6.20,PT,1.597624e+09
1,hithadhoo,-0.60,73.08,83.210,67.0,89.0,1.95,MV,1.597624e+09
2,yellowknife,62.46,-114.35,59.000,62.0,75.0,3.60,CA,1.597624e+09
3,atamanovka,51.93,113.63,53.600,100.0,90.0,6.00,RU,1.597624e+09
4,vaini,-21.20,-175.20,73.400,69.0,75.0,5.70,TO,1.597624e+09
...,...,...,...,...,...,...,...,...,...
576,mahibadhoo,3.78,72.97,82.832,72.0,100.0,1.49,MV,1.597624e+09
577,terrace bay,48.78,-87.10,61.088,81.0,56.0,2.80,CA,1.597624e+09
578,starozhilovo,54.23,39.92,60.998,79.0,0.0,4.63,RU,1.597624e+09
579,mazatlan,23.22,-106.42,89.006,66.0,90.0,2.10,MX,1.597624e+09


In [17]:
gmaps.configure(api_key=g_key)

In [18]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
Humidity = data_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
# Add the heatmap layer
fig.add_layer(heat_layer)
fig  

Figure(layout=FigureLayout(height='420px'))

In [19]:
vacation_df = data_df[(data_df["Max Temp"]<80) & (data_df["Max Temp"]>70)\
                      & (data_df["Wind Speed"]<10) & (data_df["Cloudiness"]==0)]

In [20]:
vacation_df = vacation_df.dropna()

In [21]:
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
135,casablanca,33.59,-7.62,73.004,88.0,0.0,3.10,MA,1.597624e+09
144,faanui,-16.48,-151.75,77.936,72.0,0.0,5.13,PF,1.597624e+09
252,lixourion,38.20,20.43,76.352,80.0,0.0,3.56,GR,1.597624e+09
335,mount isa,-20.73,139.50,71.600,21.0,0.0,5.70,AU,1.597624e+09
369,vaitape,-16.52,-151.75,78.944,67.0,0.0,4.76,PF,1.597624e+09


In [22]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
hotel_df = vacation_df

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found. Skipping....")                

Hotel not found. Skipping....


In [24]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
135,casablanca,33.59,-7.62,73.004,88.0,0.0,3.10,MA,1.597624e+09,Hotel Moroccan House Casablanca
144,faanui,-16.48,-151.75,77.936,72.0,0.0,5.13,PF,1.597624e+09,Bora-Bora Pearl Beach Resort & Spa
252,lixourion,38.20,20.43,76.352,80.0,0.0,3.56,GR,1.597624e+09,Summery Hotel
335,mount isa,-20.73,139.50,71.600,21.0,0.0,5.70,AU,1.597624e+09,ibis Styles Mt Isa Verona
369,vaitape,-16.52,-151.75,78.944,67.0,0.0,4.76,PF,1.597624e+09,Conrad Bora Bora Nui


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))